# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [16]:
# env_is_google_colab
# 
# True to run the script in Google Colab
# False to run the script in a local environment
# 
env_is_google_colab = True

if env_is_google_colab:
  
  print('Running in a Google Colaboratory environment.')

  from os import path
  from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
  platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
  accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
  !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
    
  from google.colab import drive
  drive.mount('/content/gdrive')
  
  print('Displaying folders ...')
  
  !ls 'gdrive/My Drive/colab/pytorch_challenge/'
    
else:
  
  print('Running in a local environment.')
  
  
# Imports here

# Main PyTorch library
import torch
# Pytorch Neural Network module
import torch.nn as nn
# Convolution functions
import torch.nn.functional as F
# Optimizatization algorithms
import torch.optim as optim

# Torchvision for image transformation and dataset loading
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision

# General definitions
if env_is_google_colab:
  base_dir = 'gdrive/My Drive/colab/pytorch_challenge'
else:
  base_dir = '/home/workspace'
  
model_checkpoint_path = base_dir + '/model_flowers.pth'
    
    
def create_model():
    
    model = models.vgg19(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    # Get the current number of input features
    num_features = model.classifier[6].in_features

    # Freeze the pre-loaded network feature parameters so we don't update it's weights
    for param in model.features.parameters():
        param.requires_grad = False

    # Remove last pre trained layer to be replaced with our classifier
    classifier = list(model.classifier.children())[:-1]
    # Adds out own classifier, with the same input features as before and 
    # our number of classes
    classifier.extend([nn.Linear(num_features, 102)])
    # Replace the model classifier
    model.classifier = nn.Sequential(*classifier)
    
    return model
  
  
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_model(file_path):

    # Force all tensors to be on CPU
    checkpoint = torch.load(file_path, map_location=lambda storage, loc: storage)
    
    model = create_model()
    
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    
    return model
  
  
  
  
# Load your model to this variable
model = load_model(model_checkpoint_path)
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

model

Running in a Google Colaboratory environment.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Displaying folders ...
 assets		    flower_data			      model_flowers.pth
 cat_to_name.json  'Image Classifier Project.ipynb'  'Test Score.ipynb'


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)